# Tutorial
This notebook serves as an tutorial to exploring the features of Pyloom, using a basic agent as an example. 
Prior to using this notebook, please ensure that you have both OpenAI and PyLoom installed.

```python
pip install openai
pip install pyloom
```

In [1]:
import copy
import inspect
import json
import os
import pdb
import re
import sys
from typing import List

# Pyloom
import pyloom as pl

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In this tutorial, we'll walk through the process of implementing a basic agent using the OpenAI library and then enhancing its functionality using Pyloom. Pyloom provides a powerful framework for tracking state of the agent and managing their states. It consists of three core components:

- **Thread** : At the heart of Pyloom lies the Thread class. Agents inherit from this class to have access to state management capabilities. To initialize a Thread object, a constructor method, often denoted as ```__init__```, must be defined within the agent's class and decorated with ```event```. This method allows the agent to establish its initial state.

- **SnapshotOnEvent** : Within the ```__init__``` method, Pyloom's SnapshotOnEvent decorator can be applied to instance variables that require tracking. When events are triggered, such variables are automatically captured in a snapshot, preserving their state at that moment. 

- **event** : By decorating methods with this decorator, the agent generates events every time the method is invoked.

In [81]:
class ChatBot(pl.Thread):
    # Event decorator for initialization
    @pl.event("Init")
    def __init__(self, system="", model="gpt-3.5-turbo"):
        super().__init__()
        self.system = system
        self.messages = []
        self.model = model
        if self.system:
            self.messages.append({"role": "system", "content": system})

        # SnapshotOnEvent decorator to capture total tokens used
        self.total_tokens = pl.SnapshotOnEvent(0)
        
        # SnapshotOnEvent decorator to capture changes in the messages list
        self.messages = pl.SnapshotOnEvent(self.messages)

    # Event decorator for user interaction
    @pl.event("Call")
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})

        # Trigger the nested 'complete' event
        return self.complete()

    # Event decorator for generating a response
    @pl.event("Complete")
    def complete(self):
        completion = openai.ChatCompletion.create(
            model=self.model,
            messages=self.messages
        )
        content = completion.choices[0].message.content
        self.total_tokens += completion.usage.total_tokens
        self.messages.append({"role": "assistant", "content": content})
        return content


In [151]:
bot = ChatBot("We have 20 apples to start with.")

In [155]:
bot("I found 10 more apples in the orchard, how many apples do we have now?")

'If you found 10 more apples in the orchard, we now have a total of 60 apples.'

In [104]:
# Show all stand alone events that are needed to reconstruct to this point
len(bot.events())

3

In [105]:
bot.events()[-1]

ThreadDecoratedEvent(event_name='Complete', event_args=(), event_kwargs={}, event_mutate_level=1)

Internally, Pyloom employs a tree data structure ```bot.tree```to manage agent states, it maintains ```bot.tree.current``` pointing to the latest data within the tree. With each event triggered, a new node is generated within this tree, capturing a snapshot of the agent's state ```bot.tree.current.data```at the time of execution. Moreover, the data structure retains not only the agent's state but also the ```args``` and ```kwargs``` passed to the specific event and a ```context``` dictionary.

In the realm of Pyloom's event hierarchy, an **event_mutate_level** of 0 signifies an event that stands alone without a parent event. When the **event_mutate_level** is 1, it represents a nested event triggered within another event. Delving deeper, an **event_mutate_level** of 2 indicates an event nested within another event that, in turn, was initiated within yet another event.

This arrangement enables Pyloom to orchestrate complex event chains and maintain a detailed record of the events.

In [106]:
# Use events to reconstruct agent from None, (0,) represents a stand alone event
copy = None 
for event in bot.events((0,)):
    copy = event.clone().mutate(copy)
copy.total_tokens

62

In [107]:
# Visualize the entire flow, 0_Call represents a stands alone event("Call"),  1_Complete represents a nested event("Complete")
copy.to_pyvis_network().show('nx.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


Each agent has **forward** and **rewind** to replay and navigate through the agent's flow.

In [200]:
bot.rewind(1) # Go back 1 step
bot.messages

[{'role': 'system', 'content': 'We have 20 apples to start with.'},
 {'role': 'user',
  'content': 'I found 10 more apples in the orchard, how many apples do we have now?'},
 {'role': 'assistant',
  'content': 'If you found 10 more apples in the orchard, we now have a total of 30 apples.'},
 {'role': 'user',
  'content': 'I found 10 more apples in the orchard, how many apples do we have now?'},
 {'role': 'assistant',
  'content': 'If you found 10 more apples in the orchard, we now have a total of 40 apples.'},
 {'role': 'user',
  'content': 'I found 10 more apples in the orchard, how many apples do we have now?'},
 {'role': 'assistant',
  'content': 'If you found 10 more apples in the orchard, we now have a total of 50 apples.'},
 {'role': 'user',
  'content': 'I found 10 more apples in the orchard, how many apples do we have now?'}]

In [201]:
# Show remain events after the current node
bot.remain_events()

[ThreadDecoratedEvent(event_name='Complete', event_args=(), event_kwargs={}, event_mutate_level=1)]

In [128]:
# Visualize the entire flow, 0_Call represents a stands alone event("Call"),  1_Complete represents a nested event("Complete")
bot.to_pyvis_network().show('nx.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [199]:
bot.rewind() # Go back all the way to the root
bot.forward() # go forward to the latest state
bot.to_pyvis_network().show('nx.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [193]:
# Replaying same events on different agent with same event names
class MockAgent(pl.Thread):
    @pl.event("Init")
    def __init__(self, system="", model="gpt-3.5-turbo"):
        print ("init...")
       
    @pl.event("Call")
    def __call__(self, message):
        print ("call")
        self.complete()

    @pl.event("Complete")
    def complete(self):
        print ("complete")

copy = None
events = bot.events((0,))
# initialize a new MockAgent
copy = events[0].mutate(copy, thread_class = MockAgent)

for event in events[1:]:
    copy = event.mutate(copy)

init...
call
complete
call
complete
call
complete
call
complete
